<div style="text-align: center; border: 2px solid #4CAF50; border-radius: 10px; padding: 20px; background-color: #f0fff0;">
    <h1 style="color: #4CAF50;">📚 Audiobook Creator Pro 🎧</h1>
    <p><b>Welcome to the enhanced Audiobook Creator! This notebook has been upgraded with advanced UI/UX features for a more powerful, intuitive, and visually appealing experience.</b></p>
</div>

---

### ✨ Key Enhancements in This Version:

**1. Polished & Themed UI:**
   - **Modern Look:** A cleaner layout with better spacing, icons, and a consistent color scheme.
   - **Visual Header:** A new title banner for a more professional feel.

**2. Enhanced Interactivity:**
   - **Cover Art Preview:** Automatically displays the book cover from `.epub` files for visual confirmation.
   - **Help Tooltips:** Click the `(?)` icons next to complex options for instant explanations.
   - **File Management:** Safely delete source books and generated audiobooks directly from the UI.

**3. Advanced Generation Controls:**
   - **Collapsible Advanced Options:** A dedicated section for fine-tuning, including Bitrate and Inter-paragraph Silence.
   - **Dynamic Button States:** The 'Generate' button now shows a spinner icon (`<i class="fa fa-spinner fa-spin"></i>`) during processing for clear, active feedback.

**4. Superior Feedback & Logging:**
   - **Color-Coded Logs:** The log window now uses colored text (e.g., <span style='color:green;'>Success</span>, <span style='color:red;'>Error</span>, <span style='color:orange;'>Warning</span>) for at-a-glance readability.
   - **Dynamic Progress Bar:** The progress bar label updates to show the current stage of the process (e.g., "Parsing Book", "Generating Audio").

---
### Instructions:
1.  **Run the cells in order.** The setup cells will automatically install new required packages like `ebooklib` and `Pillow`.
2.  The final cell will display the interactive UI.
3.  Enjoy the upgraded experience!

## Pre-Step: Clone the Repository

This first step clones the `audiobook-creator` repository from GitHub. This contains all the core Python scripts needed for the audiobook generation.

In [ ]:
import os
import subprocess

repo_url = "https://github.com/BChrisUCC/audiobook-creator"
repo_dir = "audiobook-creator"

if not os.path.exists(repo_dir):
    print(f"Cloning repository from {repo_url}...")
    try:
        # Use --depth 1 for a faster, shallow clone
        subprocess.check_call(["git", "clone", repo_url, repo_dir, "--depth", "1", "--quiet"])
        print("Repository cloned successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error cloning repository: {e}. Please ensure git is installed and you have internet access.")
    except FileNotFoundError:
        print("Error: 'git' command not found. Please ensure Git is installed and in your system's PATH.")
else:
    print(f"Repository '{repo_dir}' already exists. Skipping clone.")

## 1. Initial Setup: Configure Path & Install Packages

This cell configures the system path and installs all required Python packages. **New packages (`ebooklib`, `Pillow`) have been added** to support advanced features like cover art extraction.

In [ ]:
# Install required packages if missing and configure path
import sys
import subprocess
import os

# --- Define Repo Path ---
repo_dir = "audiobook-creator"

# --- Add Repo to Python Path ---
if os.path.isdir(repo_dir):
    sys.path.insert(0, os.path.abspath(repo_dir))
    print(f"Added '{os.path.abspath(repo_dir)}' to system path.")
else:
    print(f"Error: Directory '{repo_dir}' not found. Please run the cloning cell above.")

# --- Package Installation ---
def install_if_missing(package, import_name=None):
    if import_name is None:
        import_name = package
    try:
        __import__(import_name)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])

# List of required packages (name for pip, name for import)
required_packages = {
    "openai": "openai",
    "tqdm": "tqdm",
    "pydub": "pydub",
    "word2number": "word2number",
    "python-dotenv": "dotenv",
    "nest_asyncio": "nest_asyncio",
    "ipywidgets": "ipywidgets",
    "ebooklib": "ebooklib", # For cover art
    "Pillow": "PIL" # For image processing
}
for pkg, imp in required_packages.items():
    install_if_missing(pkg, imp)

print("All required packages are installed.")

## 2. Core Imports

This cell imports all necessary libraries, including the main processing function and new additions for handling images and EPUB files.

In [ ]:
import nest_asyncio
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, Audio, FileLink, clear_output
import os
from dotenv import load_dotenv
import asyncio
import ebooklib
from ebooklib import epub
from PIL import Image
import io

# Import the main generation function from the cloned repository.
try:
    from generate_audiobook import process_audiobook_generation
    print("Successfully imported 'process_audiobook_generation' from the cloned repository.")
except ImportError as e:
    print(f"Error: Could not import from the cloned repository. Make sure the 'audiobook-creator' directory exists and the setup cell ran correctly. Details: {e}")
    async def process_audiobook_generation(*args, **kwargs):
         yield "<p style='color:red;'><b>FATAL ERROR:</b> Could not import from repository. Cannot proceed.</p>"
         await asyncio.sleep(1)

# Apply nest_asyncio to allow nested event loops in Jupyter environments
nest_asyncio.apply()

# Define repo_dir for this cell's scope, ensuring it's available.
repo_dir = "audiobook-creator"
load_dotenv(dotenv_path=os.path.join(repo_dir, '.env'))
load_dotenv()

## 3. Widget Definitions & UI Layout

Here, we define all the interactive UI components. The layout has been completely redesigned using `HBox`, `VBox`, and `Layout` properties for a polished, well-organized interface. This includes new widgets for cover art, help tooltips, advanced options, and file management.

In [ ]:
# --- UI Styling and Layout Definitions ---
style = {'description_width': 'initial'}
layout_half = Layout(width='48%')
layout_quarter = Layout(width='23%')
layout_button = Layout(width='auto', margin='0 5px 0 5px')
help_button_layout = Layout(width='30px', margin='0 0 0 10px')

# --- Define All Widgets ---

# App Header
header = widgets.HTML("""<div style="text-align: center; border: 2px solid #4CAF50; border-radius: 10px; padding: 10px; margin-bottom:15px; background-color: #f0fff0;">
<h1 style="color: #4CAF50; margin:0;">📚 Audiobook Creator Pro 🎧</h1>
</div>""")

# Step 1: Source & Cover
repo_dir = "audiobook-creator"
book_dir = os.path.join(repo_dir, 'books')
os.makedirs(book_dir, exist_ok=True)
book_files = [f for f in os.listdir(book_dir) if f.endswith(('.epub', '.pdf', '.txt'))]
book_dropdown = widgets.Dropdown(options=book_files, description='Select Book:', value=book_files[0] if book_files else None, style=style, layout=Layout(flex='1'))
upload_widget = widgets.FileUpload(accept='.epub,.pdf,.txt', multiple=False, description='Or Upload', layout=Layout(width='180px'))
delete_book_button = widgets.Button(description='Delete Book', button_style='danger', icon='trash', layout=layout_button)
delete_book_confirm = widgets.HTML(value="")
cover_preview = widgets.Image(value=b'', format='png', width=200, height=300, layout=Layout(border='1px solid lightgray', margin='10px auto', object_fit='contain'))

# Step 2: Audio Configuration
tts_model = widgets.Dropdown(options=['kokoro', 'orpheus'], description='TTS Model:', value='kokoro', style=style)
voice_option = widgets.ToggleButtons(options=['Single Voice', 'Multi-Voice'], description='Voice Mode:', style=style)
output_format = widgets.Dropdown(options=['M4B (Chapters & Cover)', 'mp3', 'aac', 'm4a', 'wav', 'opus', 'flac', 'pcm'], description='Format:', value='M4B (Chapters & Cover)', style=style)
narrator_gender = widgets.ToggleButtons(options=['male', 'female'], description='Narrator:', style=style)
add_emotion_tags = widgets.Checkbox(value=False, description='Add Emotion Tags', indent=False)
advanced_toggle = widgets.Checkbox(value=False, description='Show Advanced Options', indent=False)

# Help Buttons & Text
help_tts = widgets.Button(description='?', tooltip='Click for info on TTS models', layout=help_button_layout)
help_emotion = widgets.Button(description='?', tooltip='Click for info on Emotion Tags', layout=help_button_layout)
help_text = widgets.HTML(value="", layout=Layout(border='1px dashed lightblue', padding='10px', margin='5px 0 10px 0', display='none'))

# Advanced Options (initially hidden)
bitrate_slider = widgets.Dropdown(options=['96k', '128k', '192k', '256k', '320k'], value='128k', description='Audio Bitrate:', style=style)
silence_slider = widgets.FloatSlider(value=0.5, min=0, max=2.0, step=0.1, description='Silence (sec):', readout_format='.1f', style=style)
advanced_options_box = widgets.VBox([bitrate_slider, silence_slider], layout=Layout(display='none', border='1px solid #ddd', padding='10px', margin_top='10px'))

# Step 3: Generation & Logs
generate_button = widgets.Button(description='Generate', button_style='success', icon='cogs', layout=layout_button)
cancel_button = widgets.Button(description='Cancel', button_style='danger', icon='stop', disabled=True, layout=layout_button)
reset_button = widgets.Button(description='Reset', button_style='warning', icon='refresh', layout=layout_button)
progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Ready.', bar_style='success', orientation='horizontal', layout=Layout(margin='5px 0'))
status_notification = widgets.HTML(value="<p><i>Status notifications will appear here.</i></p>")
log_output_html = widgets.HTML(layout={'border': '1px solid gray', 'height': '300px', 'overflow_y': 'auto', 'padding': '5px', 'margin_top': '10px', 'background_color': '#f9f9f9'})

# Step 4: Review & Download
audio_select = widgets.Dropdown(description='Audiobook:', style=style, layout=Layout(flex='1'))
play_button = widgets.Button(description='Play', button_style='info', icon='play', layout=layout_button)
download_button = widgets.Button(description='Download', button_style='primary', icon='download', layout=layout_button)
delete_audio_button = widgets.Button(description='Delete', button_style='danger', icon='trash', layout=layout_button)
delete_audio_confirm = widgets.HTML(value="")
audio_output = widgets.Output(layout=Layout(margin_top='10px'))

# --- Group Widgets into Accordion Panes ---
step1_content = widgets.VBox([
    widgets.HTML("<b>Select a book from the repo, or upload a new one.</b><br><i>Cover art preview works best with .epub files.</i>"),
    widgets.HBox([book_dropdown, upload_widget]),
    widgets.HBox([delete_book_button, delete_book_confirm]),
    cover_preview
], layout=Layout(padding='10px'))

step2_content = widgets.VBox([
    widgets.HBox([tts_model, help_tts]),
    widgets.HBox([add_emotion_tags, help_emotion]),
    help_text,
    voice_option, narrator_gender, output_format,
    widgets.HTML("<hr style='margin: 15px 0;'>"),
    advanced_toggle, advanced_options_box
], layout=Layout(padding='10px'))

step3_content = widgets.VBox([
    widgets.HBox([generate_button, cancel_button, reset_button]),
    progress_bar,
    status_notification,
    log_output_html
], layout=Layout(padding='10px'))

step4_content = widgets.VBox([
    widgets.HBox([audio_select, play_button, download_button, delete_audio_button]),
    delete_audio_confirm,
    audio_output
], layout=Layout(padding='10px'))

accordion = widgets.Accordion(children=[step1_content, step2_content, step3_content, step4_content])
accordion.set_title(0, '1. Select Source & View Cover')
accordion.set_title(1, '2. Configure Audio')
accordion.set_title(2, '3. Generate & Monitor Logs')
accordion.set_title(3, '4. Review & Download')
accordion.selected_index = 0

# --- List of widgets to manage state ---
config_widgets = [
    book_dropdown, upload_widget, delete_book_button, tts_model, voice_option,
    output_format, narrator_gender, add_emotion_tags, advanced_toggle, 
    bitrate_slider, silence_slider, help_tts, help_emotion
]

# --- Main UI Container ---
full_ui = widgets.VBox([header, accordion])

## 4. Widget Logic and Event Handlers

This section contains all the functions that respond to user interactions. It has been significantly expanded to handle the new features:
- **Cover Art Extraction:** Dynamically updates the image preview when a new book is selected.
- **Help System:** Manages the display of informational tooltips.
- **File Deletion:** Implements a two-click confirmation system to prevent accidental deletion.
- **Enhanced Logging:** A new `log_message` function formats and color-codes log entries.
- **Advanced Options:** Passes new parameters to the core generation function.

In [ ]:
# --- Global State & Helper Functions ---
is_generating = False
repo_dir = "audiobook-creator"
book_dir = os.path.join(repo_dir, 'books')
delete_book_confirm.confirm_pending = False
delete_audio_confirm.confirm_pending = False

def log_message(message, level='info'):
    """Appends a color-coded message to the HTML log widget."""
    color_map = {'info': 'black', 'success': 'green', 'warning': 'orange', 'error': 'red'}
    color = color_map.get(level, 'black')
    log_output_html.value += f"<p style='color:{color}; margin:0; font-family:monospace;'>{message}</p>"

# --- Handlers for Step 1: Source & Cover ---
def handle_upload(change):
    for filename, fileinfo in upload_widget.value.items():
        filepath = os.path.join(book_dir, filename)
        with open(filepath, 'wb') as f: f.write(fileinfo['content'])
        if filename not in list(book_dropdown.options): book_dropdown.options = sorted(list(book_dropdown.options) + [filename])
        book_dropdown.value = filename
        upload_widget.value.clear()

def update_cover_preview(change):
    """Extracts and displays cover art from an EPUB file."""
    book_path = os.path.join(book_dir, change.get('new', ''))
    if book_path and book_path.endswith('.epub') and os.path.exists(book_path):
        try:
            book = epub.read_epub(book_path)
            covers = book.get_items_of_type(ebooklib.ITEM_COVER)
            cover_item = next(covers, None)
            if cover_item:
                cover_preview.value = cover_item.get_content()
                return
        except Exception as e:
            pass # Fail silently if cover extraction fails
    # Default/placeholder image
    img = Image.new('RGB', (200, 300), color = '#f0f0f0')
    b = io.BytesIO()
    img.save(b, 'png')
    cover_preview.value = b.getvalue()

def on_delete_book_clicked(b):
    if not delete_book_confirm.confirm_pending:
        delete_book_confirm.value = "<b style='color:red;'>Click again to permanently delete.</b>"
        delete_book_confirm.confirm_pending = True
    else:
        if book_dropdown.value:
            os.remove(os.path.join(book_dir, book_dropdown.value))
            book_files = [f for f in os.listdir(book_dir) if f.endswith(('.epub', '.pdf', '.txt'))]
            book_dropdown.options = book_files
        delete_book_confirm.value = ""
        delete_book_confirm.confirm_pending = False

# --- Handlers for Step 2: Configuration ---
def on_tts_model_change(change):
    is_orpheus = change.get('new') == 'orpheus'
    add_emotion_tags.layout.display = 'flex' if is_orpheus else 'none'
    help_emotion.layout.display = 'flex' if is_orpheus else 'none'
    if not is_orpheus: add_emotion_tags.value = False

def on_advanced_toggle(change):
    advanced_options_box.layout.display = 'block' if change.get('new') else 'none'

def on_help_clicked(b):
    help_map = {
        help_tts: "<b>Kokoro:</b> Standard, high-quality voice.<br><b>Orpheus:</b> Advanced model supporting emotion tags for more expressive speech.",
        help_emotion: "<b>Emotion Tags:</b> When enabled, the script will prompt an LLM to add tags like `[laughs]` or `[sad]` to the text, which the Orpheus model can interpret for more emotive delivery."
    }
    help_text.value = help_map.get(b, "")
    help_text.layout.display = 'block'

# --- Handler for State Management ---
def set_ui_for_generation(is_active: bool):
    global is_generating
    is_generating = is_active
    for widget in config_widgets: widget.disabled = is_active
    generate_button.disabled = is_active
    reset_button.disabled = is_active
    cancel_button.disabled = not is_active
    generate_button.icon = 'spinner fa-spin' if is_active else 'cogs'

# --- Handlers for Step 3: Generation ---
def on_generate_clicked(b):
    if not book_dropdown.value: 
        status_notification.value = "<b style='color:red;'>Error: Please select or upload a book first.</b>"
        return
    set_ui_for_generation(True)
    accordion.selected_index = 2
    log_output_html.value = ""
    progress_bar.value = 0
    status_notification.value = ""
    
    log_message(f"Starting generation for: {book_dropdown.value}", 'info')
    log_message(f"Options: TTS={tts_model.value}, Mode={voice_option.value}, Format={output_format.value}", 'info')
        
    async def run_generation():
        try:
            async for update in process_audiobook_generation(
                voice_option.value, narrator_gender.value, output_format.value,
                os.path.join(book_dir, book_dropdown.value), add_emotion_tags.value,
                # Pass advanced options
                bitrate=bitrate_slider.value, 
                silence_duration=silence_slider.value
            ):
                if not is_generating:
                    log_message("Generation cancelled by user.", 'warning')
                    status_notification.value = "<b style='color:orange;'>Generation cancelled.</b>"
                    break
                
                # Log processing
                log_level = 'info'
                if "Error:" in update or "ERROR" in update: log_level = 'error'
                elif "successfully" in update or "created" in update: log_level = 'success'
                elif "Warning:" in update: log_level = 'warning'
                log_message(update, log_level)
                
                # Progress bar and status updates
                if "Progress:" in update:
                    try: 
                        percent = float(update.split("Progress:")[1].split("%")[0])
                        progress_bar.value = int(percent)
                        progress_bar.description = update.split("-")[0].strip()
                    except (ValueError, IndexError): pass
                if "Error:" in update or "ERROR" in update: status_notification.value = f"<b style='color:red;'>{update}</b>"
                if "successfully" in update or "created" in update:
                    status_notification.value = f"<b style='color:green;'>{update}</b>"
                    update_audio_file_list()
                    accordion.selected_index = 3
        except Exception as e:
            log_message(f"An unhandled error occurred: {e}", 'error')
            status_notification.value = f"<b style='color:red;'>Critical Error: {e}</b>"
        finally:
            set_ui_for_generation(False)
            progress_bar.description = "Finished."
            progress_bar.bar_style = 'success'

    asyncio.get_event_loop().run_until_complete(run_generation())

def on_cancel_clicked(b):
    global is_generating
    if is_generating: 
        is_generating = False
        status_notification.value = "<b style='color:orange;'>Cancellation signal sent. Finishing current task...</b>"

def on_reset_clicked(b):
    book_files = [f for f in os.listdir(book_dir) if f.endswith(('.epub', '.pdf', '.txt'))]
    book_dropdown.options = book_files
    book_dropdown.value = book_files[0] if book_files else None
    tts_model.value, voice_option.value, output_format.value = 'kokoro', 'Single Voice', 'M4B (Chapters & Cover)'
    narrator_gender.value, add_emotion_tags.value, advanced_toggle.value = 'male', False, False
    bitrate_slider.value, silence_slider.value = '128k', 0.5
    log_output_html.value, progress_bar.value, status_notification.value = "", 0, ""
    progress_bar.description, help_text.layout.display = "Ready.", 'none'
    delete_book_confirm.confirm_pending = False
    delete_book_confirm.value = ""

# --- Handlers for Step 4: Review & Download ---
def update_audio_file_list():
    audio_dir = 'generated_audiobooks'
    os.makedirs(audio_dir, exist_ok=True)
    try:
        audio_files = sorted([f for f in os.listdir(audio_dir)], key=lambda f: os.path.getmtime(os.path.join(audio_dir, f)), reverse=True)
        audio_select.options = audio_files
        audio_select.value = audio_files[0] if audio_files else None
    except Exception as e:
        log_message(f"Error updating file list: {e}", 'error')

def on_play_clicked(b):
    audio_output.clear_output()
    if audio_select.value: display(Audio(filename=os.path.join('generated_audiobooks', audio_select.value)))

def on_download_clicked(b):
    audio_output.clear_output()
    if audio_select.value: display(FileLink(os.path.join('generated_audiobooks', audio_select.value), result_html_prefix="<b>Download Link: </b>"))

def on_delete_audio_clicked(b):
    if not delete_audio_confirm.confirm_pending:
        delete_audio_confirm.value = "<b style='color:red;'>Click again to permanently delete.</b>"
        delete_audio_confirm.confirm_pending = True
    else:
        if audio_select.value:
            os.remove(os.path.join('generated_audiobooks', audio_select.value))
            update_audio_file_list()
        delete_audio_confirm.value = ""
        delete_audio_confirm.confirm_pending = False

# --- Register all event handlers ---
upload_widget.observe(handle_upload, names='value')
book_dropdown.observe(update_cover_preview, names='value')
delete_book_button.on_click(on_delete_book_clicked)
tts_model.observe(on_tts_model_change, names='value')
advanced_toggle.observe(on_advanced_toggle, names='value')
help_tts.on_click(on_help_clicked)
help_emotion.on_click(on_help_clicked)
generate_button.on_click(on_generate_clicked)
cancel_button.on_click(on_cancel_clicked)
reset_button.on_click(on_reset_clicked)
play_button.on_click(on_play_clicked)
download_button.on_click(on_download_clicked)
delete_audio_button.on_click(on_delete_audio_clicked)

## 5. Display the User Interface

Run this final cell to set the initial state of the UI and render the interactive Audiobook Creator Pro.

In [ ]:
# --- Set Initial States and Display UI ---
def initialize_ui():
    # Trigger handlers to set the initial UI state correctly
    on_tts_model_change({'new': tts_model.value})
    update_cover_preview({'new': book_dropdown.value})
    update_audio_file_list()
    
    # Reset confirmation flags
    delete_book_confirm.confirm_pending = False
    delete_book_confirm.value = ""
    delete_audio_confirm.confirm_pending = False
    delete_audio_confirm.value = ""

# Run initialization
initialize_ui()

# --- Display the main UI ---
print("Audiobook Creator Pro UI is ready.")
display(full_ui)